## This is an example where I applied the code for creating a csv file "smartphones.csv" that contains smartphones

In [7]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv

We will start by scraping one page and try to get the price and the product name and features related to a smartphone 

In [4]:
#Connect to website :
url='https://www.amazon.fr/Xiaomi-Smartphone-Pouces-medialek-2mp%EF%BC%8C5000mah%EF%BC%8CNoir/dp/B0DL4BKSR6/ref=sr_1_6'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

page=requests.get(url,headers=headers)

soup1=BeautifulSoup(page.content, "html.parser")
soup2=BeautifulSoup(soup1.prettify(),"html.parser")

title=soup2.find(id="productTitle").get_text(strip=True)
print(title)

#price
'''The price in this page is divided into three span elements : for the whole price, for thr fraction, for the sign $, we want to get a single element like 19,99 $'''
whole=soup2.find('span',class_='a-price-whole').get_text(strip=True)
fraction=soup2.find('span',class_='a-price-fraction').get_text(strip=True)

if whole and fraction  :
    price= whole + fraction
print(price)



Xiaomi poco c75-6 + 128gb Smartphone 6,88 Pouces 120Hz HD + écran medialek Helio g81 Triple caméra 50mp + 2mp，5000mah，Noir
78,55


Now that we wrote the code to get the name and price of a product from its page, we will open a search for smartphones on Amazon, collect all the smartphone links, and then use a for loop to go through each link and apply the code to get the product name and price. We will also go from page 1, extract the links, then move to page 2, extract the links, and so on.

# Get the products links

In [5]:
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

base_url = "https://www.amazon.fr"
search_url_prefix = "https://www.amazon.fr/s?k=smartphone&crid=1Y421Y0DYB75Q&qid=1752948329&sprefix=sma%2Caps%2C476&xpid=84qbmOTq1T7pJ&ref=sr_pg_"

product_links = [] 

for page_num in range(1, 6):  # Pages 1 to 5
    url = search_url_prefix + str(page_num)
    print(f"Scraping page {page_num}: {url}")

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    products = soup.find_all("div", {"data-component-type": "s-search-result"})

    for product in products:
        link_tag = product.find("a", class_="a-link-normal s-no-outline")
        if link_tag and "href" in link_tag.attrs:
            href = link_tag["href"]

            if "/dp/" in href:
                clean_link = base_url + href.split("?")[0]
                product_links.append(clean_link)

print(f"Found {len(product_links)} product links in total.")
for link in product_links[:10]:
    print(link)




Scraping page 1: https://www.amazon.fr/s?k=smartphone&crid=1Y421Y0DYB75Q&qid=1752948329&sprefix=sma%2Caps%2C476&xpid=84qbmOTq1T7pJ&ref=sr_pg_1
Scraping page 2: https://www.amazon.fr/s?k=smartphone&crid=1Y421Y0DYB75Q&qid=1752948329&sprefix=sma%2Caps%2C476&xpid=84qbmOTq1T7pJ&ref=sr_pg_2
Scraping page 3: https://www.amazon.fr/s?k=smartphone&crid=1Y421Y0DYB75Q&qid=1752948329&sprefix=sma%2Caps%2C476&xpid=84qbmOTq1T7pJ&ref=sr_pg_3
Scraping page 4: https://www.amazon.fr/s?k=smartphone&crid=1Y421Y0DYB75Q&qid=1752948329&sprefix=sma%2Caps%2C476&xpid=84qbmOTq1T7pJ&ref=sr_pg_4
Scraping page 5: https://www.amazon.fr/s?k=smartphone&crid=1Y421Y0DYB75Q&qid=1752948329&sprefix=sma%2Caps%2C476&xpid=84qbmOTq1T7pJ&ref=sr_pg_5
Found 240 product links in total.
https://www.amazon.fr/Xiaomi-Smartphone-Pouces-medialek-2mp%EF%BC%8C5000mah%EF%BC%8CNoir/dp/B0DL4BKSR6/ref=sr_1_6
https://www.amazon.fr/Samsung-Galaxy-64GB-Version-Internationale/dp/B0DDKLRFVV/ref=sr_1_7
https://www.amazon.fr/Samsung-Galaxy-Smartphone

# For each link, scrap the price, model and caracteristics of the products

In [13]:
# Open file once, write header
with open('smartphones.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title', 'Price in €', 'Date'])

# Now loop over product links and append data
for link in product_links:
    page_prods = requests.get(link, headers=headers)
    soup2_prods = BeautifulSoup(page_prods.content, 'html.parser')

    title_tag = soup2_prods.find(id="productTitle")
    title = title_tag.get_text(strip=True) if title_tag else "No title found"

    whole_tag = soup2_prods.find('span', class_='a-price-whole')
    fraction_tag = soup2_prods.find('span', class_='a-price-fraction')

    if whole_tag and fraction_tag:
        price = whole_tag.get_text(strip=True) + ',' + fraction_tag.get_text(strip=True)
    else:
        price = "Price not found"

    today = datetime.date.today()
    data = [title, price, today]

    # Append data inside the loop
    with open('smartphones.csv', 'a', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

